# Data Processing

In [1]:
using CSV, DataFrames, DecisionTree, Statistics, Dates, Gadfly, Random;
include("utils/precipitation.jl");

## Build features

### Get and filter the features

#### Latitude, Longitude, Height

In [2]:
features = CSV.read("data/ouvrages-surverses.csv");
colnames = ["N_Env", "ID_SOMA", "ID_OUVRAGE", "NOM", "SOMA_SEC", "REGION", "TP_X", "TP_Y", "TP_Z", "TP_LAT", "TP_LNG", "EMI_X", "EMI_Y", "EMI_LNG", "EMI_LAT"];
names!(features, Symbol.(colnames));
select!(features, [:ID_OUVRAGE, :TP_LAT, :TP_LNG, :TP_Z]);

#### Replace missing Z index with mean

In [3]:
features.TP_Z = coalesce.(features.TP_Z, mean(features[completecases(features), :].TP_Z));
first(shuffleDf(features), 10)

,ID_OUVRAGE,TP_LAT,TP_LNG,TP_Z
,String,Float64,Float64,Float64
1,0642-01D,45.6727,-73.5262,19.3526
2,3305-06D,45.6136,-73.6363,12.25
3,3410-02D,45.5325,-73.696,19.3526
4,4315-01D,45.5628,-73.5243,14.39
5,4430-06D,45.4304,-73.6655,23.67
6,3275-01D,45.637,-73.607,14.53
7,4620-04D,45.416,-73.8844,25.09
8,4795-01D,45.4499,-73.5682,15.747
9,3762-01D,45.5764,-73.6619,13.53


### Load dates and surverses

In [4]:
surverses = CSV.read("data/surverses.csv",missingstring="-99999");

#### Filter months

In [5]:
surverses = filter(row -> month(row.DATE) > 4, surverses);
surverses = filter(row -> month(row.DATE) < 11, surverses);

#### Filter non rain surverses

In [6]:
raison = coalesce.(surverses[:,:RAISON],"Inconnue");
surverses[!,:RAISON] = raison;

surverses = filter(row -> row.RAISON ∈ ["P","Inconnue","TS"], surverses);
select!(surverses, [:NO_OUVRAGE, :DATE, :SURVERSE]);

#### Remove missing data and rename

In [7]:
surverses = dropmissing(surverses, disallowmissing=true);
rename!(surverses, :NO_OUVRAGE => :ID_OUVRAGE);
first(shuffleDf(surverses),10)

,ID_OUVRAGE,DATE,SURVERSE
,String,Date,Int64
1,4280-02D,2013-06-05,0
2,4620-01D,2016-09-18,0
3,4420-04D,2018-06-17,0
4,3480-02D,2013-08-25,0
5,4430-05D,2016-07-12,0
6,4370-04D,2013-08-13,0
7,4620-01D,2018-08-03,0
8,3290-01D,2013-08-02,0
9,4430-04D,2017-05-06,0


### Augment features with dates and label

In [8]:
comb = join(features, surverses, on = :ID_OUVRAGE);
first(shuffleDf(comb), 10)

,ID_OUVRAGE,TP_LAT,TP_LNG,TP_Z,DATE,SURVERSE
,String,Float64,Float64,Float64,Date,Int64
1,3230-01D,45.6661,-73.5469,13.77,2015-07-21,0
2,3350-09D,45.5371,-73.713,23.72,2014-08-24,0
3,3490-02D,45.5054,-73.8061,24.49,2014-05-12,0
4,4270-02D,45.6112,-73.5084,19.3526,2016-10-02,0
5,4265-01D,45.626,-73.4991,11.78,2018-05-08,0
6,3350-09D,45.5371,-73.713,23.72,2013-07-12,0
7,0672-03D,45.6732,-73.5402,9.99,2013-10-12,0
8,3370-01D,45.5653,-73.6631,18.21,2013-08-11,0
9,0801-01D,45.519,-73.5275,13.71,2014-05-23,0


### Load precipitation data

#### Load and filter months between May & October included

In [9]:
precipitation = CSV.read("data/precipitations.csv",missingstring="-99999");
rename!(precipitation, Symbol("St-Hubert")=>:StHubert);

precipitation = filter(row -> month(row.date) > 4, precipitation);
precipitation = filter(row -> month(row.date) < 11, precipitation); 

#### Replace missing data by 0

In [10]:
precipitation[!,:McTavish] = coalesce.(precipitation[:,:McTavish], 0);
precipitation[!,:Bellevue] = coalesce.(precipitation[:,:Bellevue], 0);
precipitation[!,:Assomption] = coalesce.(precipitation[:,:Assomption], 0);
precipitation[!,:Trudeau] = coalesce.(precipitation[:,:Trudeau], 0);
precipitation[!,:StHubert] = coalesce.(precipitation[:,:StHubert], 0);

first(shuffleDf(precipitation), 5)

,date,heure,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64,Int64,Int64,Int64,Int64
1,2018-08-12,1,0,0,0,0,0
2,2015-10-09,19,0,0,0,0,0
3,2013-08-04,0,0,0,0,0,0
4,2014-06-24,17,0,0,100,0,5
5,2017-09-06,10,0,0,0,0,0


### Extract features from precipitation

#### Sum of precipitation for the day

In [11]:
pcp_sum = by(precipitation, :date,  McTavish = :McTavish=>sum, Bellevue = :Bellevue=>sum, 
   Assomption = :Assomption=>sum, Trudeau = :Trudeau=>sum, StHubert = :StHubert=>sum);
first(shuffleDf(pcp_sum), 5)

,date,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64,Int64,Int64,Int64
1,2019-10-01,542,0,0,548,658
2,2017-06-22,0,0,0,0,0
3,2015-10-25,204,152,167,158,178
4,2018-09-10,81,141,56,113,92
5,2017-06-07,0,0,0,0,0


#### Maximum precipitation in an hour for the day

In [12]:
pcp_max = by(precipitation, :date,  McTavish = :McTavish=>maximum, Bellevue = :Bellevue=>maximum, 
   Assomption = :Assomption=>maximum, Trudeau = :Trudeau=>maximum, StHubert = :StHubert=>maximum)
first(shuffleDf(pcp_max),5)

,date,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64,Int64,Int64,Int64
1,2015-08-01,23,17,0,47,8
2,2013-09-16,48,39,30,70,0
3,2018-06-10,0,0,2,0,0
4,2014-08-30,0,0,0,0,0
5,2019-07-27,0,0,3,0,28


### Add precipitation data to features

#### Get stations lat-lng

In [14]:
station_df = DataFrame(STATION = String[], LAT = Float64[], LNG = Float64[]);

push!(station_df, ["McTavish", 45.504742, -73.579167]);
push!(station_df, ["Bellevue", 45.427222, -73.929167]);
push!(station_df, ["Assomption", 45.809444, -73.434722]);
push!(station_df, ["Trudeau", 45.467778, -73.741667]);
push!(station_df, ["StHubert", 45.5175, -73.416944]);

station_df

,STATION,LAT,LNG
,String,Float64,Float64
1,McTavish,45.5047,-73.5792
2,Bellevue,45.4272,-73.9292
3,Assomption,45.8094,-73.4347
4,Trudeau,45.4678,-73.7417
5,StHubert,45.5175,-73.4169


### Normalize elements

#### Get extreme values

In [15]:
min_lat = min(minimum(comb.TP_LAT), minimum(station_df.LAT));
max_lat = max(maximum(comb.TP_LAT), maximum(station_df.LAT));

min_lng = min(minimum(comb.TP_LNG), minimum(station_df.LNG));
max_lng = max(maximum(comb.TP_LNG), maximum(station_df.LNG));

min_z = minimum(comb.TP_Z);
max_z = maximum(comb.TP_Z);

#### Normalize every value of comb between 0 and 1

In [16]:
comb.TP_LAT = normalize.(comb.TP_LAT, min_lat, max_lat);
comb.TP_LNG = normalize.(comb.TP_LNG, min_lng, max_lng);
comb.TP_Z = normalize.(comb.TP_Z, min_z, max_z);

first(shuffleDf(comb), 10)

,ID_OUVRAGE,TP_LAT,TP_LNG,TP_Z,DATE,SURVERSE
,String,Float64,Float64,Float64,Date,Int64
1,3520-01D,0.206585,0.17491,0.624955,2018-09-24,0
2,4795-01D,0.116487,0.719818,0.241964,2014-09-01,0
3,4370-03D,0.0842328,0.458181,0.584737,2018-08-14,0
4,0672-01D,0.687609,0.788765,0.0252755,2013-06-11,0
5,4370-03D,0.0842328,0.458181,0.584737,2014-06-04,0
6,4400-02D,0.118583,0.711983,0.255796,2016-08-08,0
7,4230-01D,0.637935,0.849315,0.112579,2013-05-24,0
8,4430-03D,0.0306819,0.604824,0.454599,2016-06-11,0
9,0801-07D,0.286683,0.783098,0.825066,2016-09-10,0


#### Normalize every value of station_df between 0 and 1

In [17]:
station_df.LAT = normalize.(station_df.LAT, min_lat, max_lat);
station_df.LNG = normalize.(station_df.LNG, min_lng, max_lng);

station_df

,STATION,LAT,LNG
,String,Float64,Float64
1,McTavish,0.251316,0.699536
2,Bellevue,0.0608415,0.0512791
3,Assomption,1.0,0.967072
4,Trudeau,0.160492,0.39856
5,StHubert,0.282664,1.0


### Augment Features

#### Add pcp_sum and pcp_max columns

In [18]:
comb.PCP_MAX = zeros(size(comb, 1));
comb.PCP_SUM = zeros(size(comb, 1));
permutecols!(comb, [:ID_OUVRAGE, :TP_LAT, :TP_LNG, :TP_Z, :DATE, :PCP_SUM, :PCP_MAX, :SURVERSE]);

In [19]:
first(shuffleDf(comb), 10)

,ID_OUVRAGE,TP_LAT,TP_LNG,TP_Z,DATE,PCP_SUM,PCP_MAX,SURVERSE
,String,Float64,Float64,Float64,Date,Float64,Float64,Int64
1,3350-03D,0.432488,0.553455,0.310401,2013-08-14,0.0,0.0,0
2,3410-02D,0.319462,0.483178,0.359859,2018-10-12,0.0,0.0,0
3,4270-02D,0.512785,0.830625,0.359859,2014-06-05,0.0,0.0,0
4,4420-02D,0.0997269,0.704644,0.285551,2016-06-22,0.0,0.0,0
5,4370-04D,0.0842328,0.458181,0.584737,2013-09-17,0.0,0.0,0
6,0642-01D,0.663919,0.797687,0.359859,2013-07-18,0.0,0.0,0
7,3350-09D,0.330728,0.451592,0.502665,2013-05-27,0.0,0.0,0
8,0801-09D,0.278466,0.780082,0.553674,2013-10-09,0.0,0.0,0
9,4720-01D,0.731445,0.86632,0.0736684,2015-08-13,0.0,0.0,0


#### Find closest station to each ouvrage and add pcp_sum and pcp_max to it

In [20]:
for i=1:size(comb, 1)
    id_ouvrage = comb[i, 1]; 
    closest_station = "McTavish"; # initial value
    shortest_dist = -1;
    
    # Find closest station
    for j=1:size(station_df, 1)
        dist = findDistance(comb[i, :TP_LAT], comb[i, :TP_LNG], station_df[j, :LAT], station_df[j, :LNG]);
        
        if shortest_dist == -1 || dist < shortest_dist
            shortest_dist = dist;
            closest_station = station_df[j, :STATION];
        end
    end
    
    # Augment comb with a weighted p_sum, based on the distance to the station
    p_sum = pcp_sum[∈([comb[i, :DATE]]).(pcp_sum.date), Symbol(closest_station)];
    comb[i, :PCP_SUM] = p_sum[1] * (1 - shortest_dist); 
    # Augment comb with a weighted p_max, based on the distance to the station
    p_max = pcp_max[∈([comb[i, :DATE]]).(pcp_max.date), Symbol(closest_station)]
    comb[i, :PCP_MAX] = p_max[1] * (1 - shortest_dist);
end

### TODO: Remove outlier in PCP_SUM and PCP_MAX that cause compression

In [30]:
comb[comb[:PCP_SUM] .> 500, :PCP_SUM] = 500;
comb[comb[:PCP_MAX] .> 500, :PCP_MAX] = 500;

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[30]:1
└ @ Core In[30]:1
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[30]:2
└ @ Core In[30]:2


In [31]:
first(shuffleDf(filter(row -> row.SURVERSE == 1, comb)), 10)

,ID_OUVRAGE,TP_LAT,TP_LNG,TP_Z,DATE,PCP_SUM,PCP_MAX,SURVERSE
,String,Float64,Float64,Float64,Date,Float64,Float64,Int64
1,3270-01D,0.586728,0.669867,0.420593,2015-10-25,135.309,43.7764,1
2,3350-01D,0.459837,0.564893,0.220482,2016-10-21,363.865,47.3625,1
3,3350-11D,0.320282,0.463777,0.553674,2014-07-31,101.772,51.2996,1
4,4340-01D,0.304064,0.76384,0.359121,2016-10-20,188.867,54.093,1
5,0801-05D,0.282104,0.788458,0.142334,2017-06-30,44.3891,40.7655,1
6,3260-01D,0.609996,0.697487,0.386587,2013-09-02,137.883,80.1643,1
7,4310-01D,0.398976,0.803102,0.182552,2013-07-17,196.714,196.714,1
8,3250-01D,0.623463,0.710794,0.288167,2015-09-19,48.9592,34.5225,1
9,4430-05D,0.0529114,0.554875,0.729588,2017-07-14,62.3887,34.8404,1


#### Normalize pcp_sum and pcp_max

In [32]:
min_pcp_sum = minimum(comb.PCP_SUM);
max_pcp_sum = maximum(comb.PCP_SUM);

min_pcp_max = minimum(comb.PCP_MAX);
max_pcp_max = maximum(comb.PCP_MAX);

In [33]:
comb.PCP_SUM = normalize.(comb.PCP_SUM, min_pcp_sum, max_pcp_sum);
comb.PCP_MAX = normalize.(comb.PCP_MAX, min_pcp_max, max_pcp_max);

In [34]:
first(shuffleDf(filter(row -> row.SURVERSE == 1, comb)), 10)

,ID_OUVRAGE,TP_LAT,TP_LNG,TP_Z,DATE,PCP_SUM,PCP_MAX,SURVERSE
,String,Float64,Float64,Float64,Date,Float64,Float64,Int64
1,3370-01D,0.400013,0.544048,0.322499,2017-08-22,0.227608,0.16011,1
2,3410-02D,0.319462,0.483178,0.359859,2013-08-13,0.178741,0.109868,1
3,4380-01D,0.160421,0.728206,0.359859,2016-10-21,0.875741,0.113991,1
4,4430-05D,0.0529114,0.554875,0.729588,2016-10-27,0.202561,0.0518555,1
5,3750-01D,0.506656,0.587505,0.106366,2017-10-09,0.467314,0.118271,1
6,4230-04D,0.660589,0.861562,0.0200438,2018-06-18,0.413812,0.170166,1
7,0801-05D,0.282104,0.788458,0.142334,2013-06-29,0.0,0.0,1
8,3260-01D,0.609996,0.697487,0.386587,2016-10-21,0.620792,0.0808056,1
9,3290-01D,0.543825,0.620308,0.376124,2015-05-11,0.289932,0.0641195,1


#### Normalize dates (UNUSED ANYMORE)

In [35]:
dates = dateToDay.(comb.DATE);
dates = normalize.(dates, minimum(dates), maximum(dates));

# Model creation

### Separate features and labels

In [66]:
train_features = convert(Matrix{Float64},comb[:, [:TP_LAT, :TP_LNG, :TP_Z, :PCP_SUM, :PCP_MAX]]);
# train_features = hcat(dates, train_features);

In [37]:
train_labels = comb[:, :SURVERSE];

### Build Model

#### Test with tree first

In [38]:
model_tree = build_tree(train_labels, train_features)

Decision Tree
Leaves: 3793
Depth:  38

In [39]:
model_tree = prune_tree(model_tree, 0.90)

Decision Tree
Leaves: 3545
Depth:  38

In [44]:
model = build_forest(train_labels, train_features, 3, 30, 0.5, 10)

Ensemble of Decision Trees
Trees:      30
Avg Leaves: 357.56666666666666
Avg Depth:  10.0

# Prediction

## Get the test data

In [45]:
test = CSV.read("data/test.csv");
rename!(test, :NO_OUVRAGE => :ID_OUVRAGE);
first(test, 10)

,ID_OUVRAGE,DATE
,String,Date
1,3260-01D,2019-05-02
2,3260-01D,2019-05-09
3,3260-01D,2019-05-10
4,3260-01D,2019-05-15
5,3260-01D,2019-05-20
6,3260-01D,2019-05-23
7,3260-01D,2019-05-24
8,3260-01D,2019-05-26
9,3260-01D,2019-05-30


In [46]:
to_merge = unique(comb[!, [:ID_OUVRAGE, :TP_LAT, :TP_LNG, :TP_Z]], :ID_OUVRAGE);
test_comb = join(test, to_merge, on= [:ID_OUVRAGE]);
nrow(test_comb)

283

In [47]:
first(shuffleDf(test_comb), 10);

### Add PCP_SUM and PCP_MAX

#### Initialize default pcp

In [48]:
test_comb.PCP_MAX = zeros(size(test_comb, 1));
test_comb.PCP_SUM = zeros(size(test_comb, 1));
permutecols!(test_comb, [:ID_OUVRAGE, :TP_LAT, :TP_LNG, :TP_Z, :DATE, :PCP_SUM, :PCP_MAX]);

In [49]:
first(shuffleDf(test_comb), 10);

#### Populate pcp

In [50]:
for i=1:size(test_comb, 1)
    id_ouvrage = test_comb[i, 1]; 
    closest_station = "McTavish"; # initial value
    shortest_dist = -1;
    
    # Find closest station
    for j=1:size(station_df, 1)
        dist = findDistance(test_comb[i, :TP_LAT], test_comb[i, :TP_LNG], station_df[j, :LAT], station_df[j, :LNG]);
        
        if shortest_dist == -1 || dist < shortest_dist
            shortest_dist = dist;
            closest_station = station_df[j, :STATION];
        end
    end
    
    # Augment comb with a weighted p_sum, based on the distance to the station
    p_sum = pcp_sum[∈([test_comb[i, :DATE]]).(pcp_sum.date), Symbol(closest_station)];
    test_comb[i, :PCP_SUM] = p_sum[1] * (1 - shortest_dist); 
    # Augment comb with a weighted p_max, based on the distance to the station
    p_max = pcp_max[∈([test_comb[i, :DATE]]).(pcp_max.date), Symbol(closest_station)]
    test_comb[i, :PCP_MAX] = p_max[1] * (1 - shortest_dist);
end

In [51]:
first(shuffleDf(test_comb), 10)

,ID_OUVRAGE,TP_LAT,TP_LNG,TP_Z,DATE,PCP_SUM,PCP_MAX
,String,Float64,Float64,Float64,Date,Float64,Float64
1,3260-01D,0.609996,0.697487,0.386587,2019-07-11,241.776,132.752
2,4380-01D,0.160421,0.728206,0.359859,2019-05-30,6.33284,6.33284
3,3260-01D,0.609996,0.697487,0.386587,2019-07-14,7.69577,5.13052
4,4240-01D,0.607546,0.868856,0.116503,2019-07-06,31.1831,29.8838
5,3350-07D,0.353054,0.490395,0.405552,2019-08-26,1.57332,1.57332
6,4350-01D,0.237379,0.74432,0.359859,2019-08-29,0.0,0.0
7,4380-01D,0.160421,0.728206,0.359859,2019-08-17,175.51,157.416
8,3260-01D,0.609996,0.697487,0.386587,2019-07-01,0.0,0.0
9,4240-01D,0.607546,0.868856,0.116503,2019-09-15,0.0,0.0


#### Normalize pcp based on comb (not test_comb)

In [61]:
min_pcp_sum = minimum(comb.PCP_SUM);
max_pcp_sum = maximum(comb.PCP_SUM);

min_pcp_max = minimum(comb.PCP_MAX);
max_pcp_max = maximum(comb.PCP_MAX);

In [62]:
test_comb.PCP_SUM = normalize.(test_comb.PCP_SUM, min_pcp_sum, max_pcp_sum);
test_comb.PCP_MAX = normalize.(test_comb.PCP_MAX, min_pcp_max, max_pcp_max);

In [64]:
first(test_comb, 20)

,ID_OUVRAGE,TP_LAT,TP_LNG,TP_Z,DATE,PCP_SUM,PCP_MAX
,String,Float64,Float64,Float64,Date,Float64,Float64
1,3260-01D,0.609996,0.697487,0.386587,2019-05-02,0.0454408,0.0390072
2,3260-01D,0.609996,0.697487,0.386587,2019-05-09,0.155547,0.146277
3,3260-01D,0.609996,0.697487,0.386587,2019-05-10,0.672874,0.214539
4,3260-01D,0.609996,0.697487,0.386587,2019-05-15,0.00349545,0.00650119
5,3260-01D,0.609996,0.697487,0.386587,2019-05-20,0.0803953,0.130024
6,3260-01D,0.609996,0.697487,0.386587,2019-05-23,0.305852,0.204788
7,3260-01D,0.609996,0.697487,0.386587,2019-05-24,0.0227204,0.0292554
8,3260-01D,0.609996,0.697487,0.386587,2019-05-26,0.00524317,0.00975179
9,3260-01D,0.609996,0.697487,0.386587,2019-05-30,0.0122341,0.0227542


#### Normalize dates (UNUSED ANYMORE)

In [203]:
test_dates = dateToDay.(test_comb.DATE);
test_dates = normalize.(test_dates, minimum(test_dates), maximum(test_dates));

### Create Test features

In [63]:
test_features = convert(Matrix{Float64}, test_comb[:, [:TP_LAT, :TP_LNG, :TP_Z, :PCP_SUM, :PCP_MAX]]);
# test_features = hcat(test_dates, test_features);

test_features

283×5 Array{Float64,2}:
 0.609996  0.697487  0.386587  0.0454408   0.0390072 
 0.609996  0.697487  0.386587  0.155547    0.146277  
 0.609996  0.697487  0.386587  0.672874    0.214539  
 0.609996  0.697487  0.386587  0.00349545  0.00650119
 0.609996  0.697487  0.386587  0.0803953   0.130024  
 0.609996  0.697487  0.386587  0.305852    0.204788  
 0.609996  0.697487  0.386587  0.0227204   0.0292554 
 0.609996  0.697487  0.386587  0.00524317  0.00975179
 0.609996  0.697487  0.386587  0.0122341   0.0227542 
 0.353054  0.490395  0.405552  0.10076     0.0837334 
 0.353054  0.490395  0.405552  0.0278697   0.0318984 
 0.353054  0.490395  0.405552  0.0         0.0       
 0.353054  0.490395  0.405552  0.184369    0.171454  
 ⋮                                                   
 0.160421  0.728206  0.359859  0.0         0.0       
 0.160421  0.728206  0.359859  0.86785     0.279719  
 0.160421  0.728206  0.359859  0.559665    0.577781  
 0.160421  0.728206  0.359859  0.0         0.0       
 0.1

## Predict

In [67]:
test_labels = apply_forest(model, test_features)

283-element Array{Int64,1}:
 0
 0
 1
 0
 0
 1
 0
 0
 0
 0
 0
 0
 1
 ⋮
 0
 1
 1
 0
 0
 0
 0
 0
 0
 0
 0
 0

## Generate submission

In [68]:
ID = test_comb[:,:ID_OUVRAGE].*"_".*string.(test_comb[:,:DATE])
sampleSubmission = DataFrame(ID = ID, Surverse=test_labels)
CSV.write("submissions/mc-submission-5.csv",sampleSubmission)

"submissions/mc-submission-5.csv"